In [2]:
from graph2text.finetune import SummarizationModule, Graph2TextModule
import argparse
import pytorch_lightning as pl
import os
import sys
from pathlib import Path
import pdb

SEED = 42

In [1]:
import torch
torch.cuda.is_available(), torch.cuda.device_count()

(True, 2)

In [3]:
#MODEL='t5-base'
DATA_DIR = './graph2text/data/webnlg'
OUTPUT_DIR = './graph2text/outputs/port_test'
CHECKPOINT = './graph2text/outputs/t5-base_13881/val_avg_bleu=68.1000-step_count=5.ckpt'
MAX_LENGTH = 384

In [4]:
#parser = argparse.ArgumentParser()
#parser = pl.Trainer.add_argparse_args(parser)
#parser = SummarizationModule.add_model_specific_args(parser, os.getcwd())

In [6]:
#args = parser.parse_args([
#    '--data_dir',DATA_DIR,
#    '--task','graph2text',
#    '--model_name_or_path',MODEL,
#    '--eval_batch_size','8',
#    '--gpus','1',
#    '--output_dir',OUTPUT_DIR,
#    '--checkpoint',CHECKPOINT,
#    '--max_source_length','384',
#    '--max_target_length','384',
#    '--val_max_target_length','384',
#    '--test_max_target_length','384',
#    '--eval_max_gen_length','384',
#    '--do_predict',
#    '--eval_beams','3'
#])

In [7]:
override_args = {
    'data_dir': DATA_DIR,
    'output_dir': OUTPUT_DIR,
}

In [8]:
model_ckp = Graph2TextModule.load_from_checkpoint(CHECKPOINT, strict=False, **override_args)

Graph2Text hparams are: Namespace(logger=True, checkpoint_callback=True, early_stop_callback=False, default_root_dir=None, gradient_clip_val=0, process_position=0, num_nodes=1, num_processes=1, gpus=1, auto_select_gpus=False, log_gpu_memory=None, progress_bar_refresh_rate=1, overfit_batches=0.0, track_grad_norm=-1, check_val_every_n_epoch=1, fast_dev_run=False, accumulate_grad_batches=1, max_epochs=100, min_epochs=1, max_steps=None, min_steps=None, limit_train_batches=1.0, limit_val_batches=1.0, limit_test_batches=1.0, val_check_interval=1.0, log_save_interval=100, row_log_interval=50, distributed_backend=None, sync_batchnorm=False, precision=32, weights_summary='top', weights_save_path=None, num_sanity_val_steps=2, truncated_bptt_steps=None, resume_from_checkpoint=None, profiler=None, benchmark=False, deterministic=False, reload_dataloaders_every_epoch=False, auto_lr_find=False, replace_sampler_ddp=True, terminate_on_nan=False, auto_scale_batch_size=False, prepare_data_per_node=True, 

We have added 3 tokens
parameters Namespace(logger=True, checkpoint_callback=True, early_stop_callback=False, default_root_dir=None, gradient_clip_val=0, process_position=0, num_nodes=1, num_processes=1, gpus=1, auto_select_gpus=False, log_gpu_memory=None, progress_bar_refresh_rate=1, overfit_batches=0.0, track_grad_norm=-1, check_val_every_n_epoch=1, fast_dev_run=False, accumulate_grad_batches=1, max_epochs=100, min_epochs=1, max_steps=None, min_steps=None, limit_train_batches=1.0, limit_val_batches=1.0, limit_test_batches=1.0, val_check_interval=1.0, log_save_interval=100, row_log_interval=50, distributed_backend=None, sync_batchnorm=False, precision=32, weights_summary='top', weights_save_path=None, num_sanity_val_steps=2, truncated_bptt_steps=None, resume_from_checkpoint=None, profiler=None, benchmark=False, deterministic=False, reload_dataloaders_every_epoch=False, auto_lr_find=False, replace_sampler_ddp=True, terminate_on_nan=False, auto_scale_batch_size=False, prepare_data_per_n

DEALING WITH UNKNOWN TOKENS
1. See what words have characters outside the vocab
2. replace these chars with </unk> (??)
3. create a mapping (like Taej ?? n to Taejŏn)
4. Map them back together in the sentence (if it has Taej ?? n, replace with Taejŏn)


In [10]:
#model_ckp.tokenizer.__dict__

In [14]:
import time

inputs = [
    'translate Graph to English: <H> World Trade Center <R> height <T> 200 meter <H> World Trade Center <R> is a <T> tower',
]

inputs_encoding = model_ckp.tokenizer(
    inputs, truncation=True, max_length=MAX_LENGTH, return_tensors='pt'
)

print(inputs_encoding['input_ids'])

now = time.time()

model_ckp.model.eval()
with torch.no_grad():
    gen_output = model_ckp.model.generate(
        inputs_encoding['input_ids'],
        attention_mask=inputs_encoding['attention_mask'],
        use_cache=True,
        decoder_start_token_id = model_ckp.decoder_start_token_id,
        num_beams=model_ckp.eval_beams,
        max_length=model_ckp.eval_max_length,
        length_penalty=1.0    
    )

print([model_ckp.tokenizer.decode(i) for i in inputs_encoding['input_ids']])
print([model_ckp.tokenizer.decode(i, skip_special_tokens=True) for i in gen_output])
print(time.time() - now)

tensor([[13959,     3, 21094,    12,  1566,    10, 32100,  1150,  6550,  1166,
         32101,  3902, 32102,  2382,     3,  4401, 32100,  1150,  6550,  1166,
         32101,    19,     3,     9, 32102,  7293,     1]])
['translate Graph to English:<H> World Trade Center<R> height<T> 200 meter<H> World Trade Center<R> is a<T> tower</s>']
['The World Trade Center is a tower with a height of 200 meters.']
1.7831895351409912


In [9]:
import re
vocab = model_ckp.tokenizer.get_vocab()
convert_some_japanese_characters = True
N = 2
class UnknownCharReplacer():
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.vocab = tokenizer.get_vocab()
        self.unknowns = []
        
    def read_label(self, label):
        self.unknowns.append({})
        
        # Some pre-processing of labels to normalise some characters
        if convert_some_japanese_characters:
            label = label.replace('（','(')
            label = label.replace('）',')')
            label = label.replace('〈','<')
            label = label.replace('／','/')
            label = label.replace('〉','>')
        
        
        label_encoded = self.tokenizer.encode(label)
        label_tokens = self.tokenizer.convert_ids_to_tokens(label_encoded)
        label_token_to_string = self.tokenizer.convert_tokens_to_string(label_tokens)
        unk_token_to_string = model_ckp.tokenizer.convert_tokens_to_string([model_ckp.tokenizer.unk_token])
                
        #print(label_encoded,label_tokens,label_token_to_string)
        
        match_unks_in_label = re.findall('(?:(?: )*⁇(?: )*)+', label_token_to_string)
        if len(match_unks_in_label) > 0:
            # If the whole label is made of UNK
            if match_unks_in_label[0] == label_token_to_string:
                #print('Label is all unks')
                self.unknowns[-1][label_token_to_string.strip()] = label
            # Else, there should be non-UNK characters in the label
            else:
                #print('Label is NOT all unks')
                # Analyse the label with a sliding window of size N (N before, N ahead)
                for idx, token in enumerate(label_tokens):
                    idx_before = max(0,idx-N)
                    idx_ahead = min(len(label_tokens), idx+N+1)
                    
                                       
                    # Found a UNK
                    if token == self.tokenizer.unk_token:
                        
                        # In case multiple UNK, exclude UNKs seen after this one, expand window to other side if possible
                        if len(match_unks_in_label) > 1:
                            #print(idx)
                            #print(label_tokens)
                            #print(label_tokens[idx_before:idx_ahead])
                            #print('HERE!')
                            # Reduce on the right, expanding on the left
                            while model_ckp.tokenizer.unk_token in label_tokens[idx+1:idx_ahead]:
                                idx_before = max(0,idx_before-1)
                                idx_ahead = min(idx+2, idx_ahead-1)
                                #print(label_tokens[idx_before:idx_ahead])
                            # Now just reduce on the left
                            while model_ckp.tokenizer.unk_token in label_tokens[idx_before:idx]:
                                idx_before = min(idx-1,idx_before+2)
                                #print(label_tokens[idx_before:idx_ahead])
                                
                        # First token of the label is UNK
                        span = self.tokenizer.convert_tokens_to_string(label_tokens[idx_before:idx_ahead])
                        if idx == 1 and label_tokens[0] == '▁':
                            #print('Label begins with unks')
                            to_replace = '^' + re.escape(span).replace(
                                    re.escape(unk_token_to_string),
                                    '.+?'
                                )
                            
                            replaced_span = re.search(
                                to_replace,
                                label
                            )[0]
                            self.unknowns[-1][span.strip()] = replaced_span
                        # Last token of the label is UNK
                        elif idx == len(label_tokens)-2 and label_tokens[-1] == model_ckp.tokenizer.eos_token:
                            #print('Label ends with unks')
                            pre_idx = self.tokenizer.convert_tokens_to_string(label_tokens[idx_before:idx])
                            pre_idx_unk_counts = pre_idx.count(unk_token_to_string)
                            to_replace = re.escape(span).replace(
                                    re.escape(unk_token_to_string),
                                    f'[^{re.escape(pre_idx)}]+?'
                                ) + '$'
                            
                            if pre_idx.strip() == '':
                                to_replace = to_replace.replace('[^]', '(?<=\s)[^a-zA-Z0-9]')
                            
                            replaced_span = re.search(
                                to_replace,
                                label
                            )[0]
                            self.unknowns[-1][span.strip()] = replaced_span
                            
                        # A token in-between the label is UNK                            
                        else:
                            #print('Label has unks in the middle')
                            pre_idx = self.tokenizer.convert_tokens_to_string(label_tokens[idx_before:idx])

                            to_replace = re.escape(span).replace(
                                re.escape(unk_token_to_string),
                                f'[^{re.escape(pre_idx)}]+?'
                            )
                            #If there is nothing behind the ??, because it is in the middle but the previous token is also
                            #a ??, then we would end up with to_replace beginning with [^], which we can't have
                            if pre_idx.strip() == '':
                                to_replace = to_replace.replace('[^]', '(?<=\s)[^a-zA-Z0-9]')
        
                            replaced_span = re.search(
                                to_replace,
                                label
                            )
                            
                            if replaced_span:
                                span = re.sub(r'\s([?.!",](?:\s|$))', r'\1', span.strip())
                                self.unknowns[-1][span] = replaced_span[0]

                    
        
    def replace_on_sentence(self, sentence):
        # Loop through in case the labels are repeated, maximum of three times
        loop_n = 3
        while '⁇' in sentence and loop_n > 0:
            loop_n -= 1
            for unknowns in self.unknowns:
                for k,v in unknowns.items():
                    # In case it is because the first letter of the sentence has been uppercased
                    if not k in sentence and k[0] == k[0].lower() and k[0].upper() == sentence[0]:
                        k = k[0].upper() + k[1:]
                        v = v[0].upper() + v[1:]
                    # In case it is because a double space is found where it should not be
                    elif not k in sentence and len(re.findall(r'\s{2,}',k))>0:
                        k = re.sub(r'\s+', ' ', k)
                    #print(k,'/',v,'/',sentence)
                    sentence = sentence.replace(k.strip(),v.strip(),1)
                    #sentence = re.sub(k, v, sentence)
            sentence = re.sub(r'\s+', ' ', sentence).strip()
            sentence = re.sub(r'\s([?.!",](?:\s|$))', r'\1', sentence)
        return sentence
    
replacer = UnknownCharReplacer(model_ckp.tokenizer)
replacer.read_label('Cuhppulčohkka')
replacer.read_label('Cuhppulčohkka')
replacer.replace_on_sentence('Cuhppul ⁇ ohkka is a native label.'), replacer.unknowns

('The directions of Khachkar are Գողգոթավանքից.', [{}, {'⁇': 'Գողգոթավանքից'}])

# STRATEGY TO REPLACE ??

- TREAT CASE WHERE THE WHOLE LABEL IS MADE OF UNKNOWNS

- IF IT ISNT, THERE MUST BE SPACE BEFORE AND AFTER THAT IS KNOWN
     - GROUP UNKNOWNS INTO CONTINUOUS UNK TOKENS (?? ?? -> ??)
    

    - LOOK TO SEE IF IT IS IN THE BEGINNING OF THE SENTENCE
        - TREAT CASE
    - LOOK TO SEE IF IT IS IN THE ENDING OF THE SENTENCE
        - TREAT CASE
    - IT IS IN THE MIDDLE, TREAT AS NORMAL

In [10]:
import pandas as pd
df = pd.read_csv('sampled_df_pre_verbalisation.csv')
df_sample = df.sample(64, random_state=SEED).reset_index(drop=True)

In [11]:
# Dataset and Dataloader
from torch.utils.data import Dataset, DataLoader

class TripleLabelDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.len = self.df.shape[0]
    def __getitem__(self, index):      
        row = self.df.iloc[index]
        item = f"translate Graph to English: <H> {row['entity_label']} <R> {row['property_label']} <T> {row['object_label']}"
        #return model_ckp.tokenizer.prepare_seq2seq_batch(
        #    item, max_length=args.max_source_length, return_tensors='pt'
        #)
        return item
    def __len__(self):
        return self.len    

In [12]:
# Pilot Sample
sample_data = TripleLabelDataset(df_sample)
sample_dataloader = DataLoader(dataset=sample_data, batch_size=64)
print(len(sample_dataloader))

# Full Data
data = TripleLabelDataset(df)
dataloader = DataLoader(dataset=data, batch_size=16)
print(len(dataloader))

1
476


In [13]:
def replace_verbalisation_on_df(row):
    try:
        replacer = UnknownCharReplacer(model_ckp.tokenizer)
        replacer.read_label(row['entity_label'])
        replacer.read_label(row['object_label'])
        return replacer.replace_on_sentence(row['verbalisation'])
    except Exception:
        print(row)
        raise

## Pilot Sample

In [ ]:
import time

verbalisations = []
start_idx = 0
for idx, batch in enumerate(sample_dataloader):
    if idx < start_idx:
        print(f'Skipping idx {idx}')
        continue
    print(idx,end=': ')
    
    inputs_encoding = model_ckp.tokenizer.prepare_seq2seq_batch(
        batch, max_length=args.max_source_length, return_tensors='pt'
    )
    
    now = time.monotonic()
    model_ckp.model.eval()
    with torch.no_grad():
        gen_output = model_ckp.model.generate(
            inputs_encoding['input_ids'],
            attention_mask=inputs_encoding['attention_mask'],
            use_cache=True,
            decoder_start_token_id = model_ckp.decoder_start_token_id,
            num_beams=model_ckp.eval_beams,
            max_length=model_ckp.eval_max_length,
            length_penalty=1.0
        )
    print('Generated batch in', time.strftime("%H:%M:%S", time.gmtime(time.monotonic() - now)))
    
    verbalisations = verbalisations + [model_ckp.tokenizer.decode(i) for i in gen_output]
  
    start_idx += 1
    
    #break

In [89]:
df_sample['verbalisation'] = verbalisations
df_sample['verbalisation'] = df_sample['verbalisation'].apply(lambda x : x[0].upper() + x[1:])
df_sample['processed_verbalisation'] = df_sample.apply(replace_verbalisation_on_df ,axis=1)
df_sample.to_csv('pilot_sampled_df_verbalised.csv', index=None)

## Full Data

In [ ]:
import time

start_idx = 0
for idx, batch in enumerate(dataloader):
    if idx < start_idx:
        print(f'Skipping idx {idx}')
        continue
    print(idx,end=': ')
    
    inputs_encoding = model_ckp.tokenizer.prepare_seq2seq_batch(
        batch, max_length=args.max_source_length, return_tensors='pt'
    )
    
    
    now = time.monotonic()
    model_ckp.model.eval()
    with torch.no_grad():
        gen_output = model_ckp.model.generate(
            inputs_encoding['input_ids'],
            attention_mask=inputs_encoding['attention_mask'],
            use_cache=True,
            decoder_start_token_id = model_ckp.decoder_start_token_id,
            num_beams=model_ckp.eval_beams,
            max_length=model_ckp.eval_max_length,
            length_penalty=1.0
        )
    print('Generated batch in', time.strftime("%H:%M:%S", time.gmtime(time.monotonic() - now)))
    
    verbalisations = [model_ckp.tokenizer.decode(i) for i in gen_output]

    with open(f'verbalisations/verbalisations_batch_{idx}.txt','w+') as f:
        for v in verbalisations:
            f.write(v)
            f.write('\n')    
    
    start_idx += 1

0: Generated batch in 00:03:15
1: Generated batch in 00:05:38
2: Generated batch in 00:06:20
3: Generated batch in 00:05:50
4: Generated batch in 00:05:02
5: Generated batch in 00:03:37
6: Generated batch in 00:04:07
7: Generated batch in 00:05:52
8: Generated batch in 00:04:05
9: Generated batch in 00:04:51
10: Generated batch in 00:06:57
11: Generated batch in 00:04:52
12: Generated batch in 00:03:45
13: Generated batch in 00:04:53
14: Generated batch in 00:04:37
15: Generated batch in 00:05:03
16: Generated batch in 00:03:05
17: Generated batch in 00:04:07
18: Generated batch in 00:05:04
19: Generated batch in 00:04:53
20: Generated batch in 00:04:53
21: Generated batch in 00:04:08
22: Generated batch in 00:03:37
23: Generated batch in 00:05:12
24: Generated batch in 00:04:17
25: Generated batch in 00:04:39
26: Generated batch in 00:03:48
27: Generated batch in 00:04:27
28: Generated batch in 00:03:11
29: Generated batch in 00:03:58
30: Generated batch in 00:06:03
31: Generated batc

In [14]:
# Collect all verbalisations from .txt files
import glob
n_filenames = len(glob.glob('verbalisations/*.txt'))
verbalisations = []

for idx in range(n_filenames):
    filename = f'verbalisations/verbalisations_batch_{idx}.txt'
    with open(filename,'r') as f:
        for line in f:
            verbalisations.append(line.strip())

In [15]:
df['verbalisation'] = verbalisations
df['verbalisation'] = df['verbalisation'].apply(lambda x : x[0].upper() + x[1:])
df['processed_verbalisation'] = df.apply(replace_verbalisation_on_df ,axis=1)
df['unk_count'] = df['verbalisation'].apply(lambda x : x.count('⁇'))

# Check if verbs are the same for unk = 0
df[df['verbalisation'] == df['processed_verbalisation']].equals(df[df['unk_count'] == 0])

True

In [16]:
# First drop those without labels

df = df[(df['entity_label_lan'] != 'none') & (df['property_label_lan'] != 'none') & (df['object_label_lan'] != 'none')].reset_index(drop=True)
# Create a new id to stratify per property AND theme
df['property_and_theme_id'] = df.apply(lambda x : x['property_id'] + x['theme_entity_id'], axis=1)


# Then select those in English
df_english = df[(df['entity_label_lan'] == 'en') & (df['property_label_lan'] == 'en') & (df['object_label_lan'] == 'en')].reset_index(drop=True)

# Create a group indication column (strata) for splitting (due to fund available we can only analyse a portion)
df_english['campaign_group'] = -1

In [31]:
from sklearn.model_selection import StratifiedKFold as StratifiedKFold
skf = StratifiedKFold(n_splits=42, shuffle=True, random_state=42)

for idx, (train_index, test_index) in enumerate(skf.split(df_english, df_english['theme_entity_id'])):
    df_english.loc[test_index, 'campaign_group'] = idx
    
# Number of examples per group
df_english['campaign_group'].value_counts()[0]

159

In [33]:
# Number of examples per group
df_english[df_english['campaign_group'] == 0]['theme_entity_id'].value_counts()

Q1248784     9
Q2066131     9
Q16521       9
Q6999        9
Q12973014    9
Q483501      9
Q11173       9
Q334166      8
Q2095        8
Q3918        8
Q515         8
Q11631       8
Q1114461     8
Q8502        8
Q82955       8
Q79007       8
Q41176       7
Q47461344    7
Q4989906     5
Q3305213     5
Name: theme_entity_id, dtype: int64

In [22]:
df.to_csv('campaign_sampled_df_verbalised.csv', index=None)
df_english.to_csv('campaign_sampled_df_verbalised_english.csv', index=None)

In [48]:
df_english[df_english['campaign_group'] == 0][['entity_label','property_label','object_label','verbalisation','processed_verbalisation']]

,entity_label,property_label,object_label,verbalisation,processed_verbalisation
28,Tabubil Airport,located in time zone,UTC+10:00,Tabubil Airport is located in the time zone UT...,Tabubil Airport is located in the time zone UT...
58,Reus Airport,has part,Reial Aero Club de Reus,Reial Aero Club de Reus is part of Reus Airport.,Reial Aero Club de Reus is part of Reus Airport.
69,Embaba Airport,"dissolved, abolished or demolished date",2002,"Embaba Airport was dismantled, abolished or de...","Embaba Airport was dismantled, abolished or de..."
94,Utti Airport,location,Utti,Utti Airport is located in Utti.,Utti Airport is located in Utti.
142,Leicester Airport,historic county,Leicestershire,Leicester Airport is located in the historic c...,Leicester Airport is located in the historic c...
...,...,...,...,...,...
7368,Kodra e Robit,located in time zone,UTC+01:00,Kodra e Robit is located in the time zone UTC+...,Kodra e Robit is located in the time zone UTC+...
7378,Hochwang,parent peak,Aroser Rothorn,Aroser Rothorn is the parent peak of Hochwang.,Aroser Rothorn is the parent peak of Hochwang.
7460,Stoppelsberg,part of,Fulda-Haune-Tafelland,Stoppelsberg is part of Fulda-Haune-Tafelland.,Stoppelsberg is part of Fulda-Haune-Tafelland.
7499,Cuhppulčohkka,native label,Cuhppulčohkka,Cuhppul ⁇ ohkka is a native label.,Cuhppulčohkka is a native label.


In [788]:
#df_unk_1 = df[df['unk_count'] == 1].copy()
#df_unk_1['processed_verbalisation'] = df_unk_1.apply(replace_verbalisation_on_df ,axis=1)
#
##df_unk_1_bk = df_unk_1.copy()
#print(df_unk_1.equals(df_unk_1_bk))
##df_unk_1[['entity_label','object_label','processed_verbalisation']].to_csv('test.csv')

#df_unk_1[df_unk_1['processed_verbalisation'] != df_unk_1_bk['processed_verbalisation']][['entity_label','object_label','verbalisation','processed_verbalisation']]

In [30]:
import time

inputs = [
    '''translate Graph to English: 
    <H> antipasto <R> aspect of <T> Italian cuisine 
    ''',
]

inputs_encoding = model_ckp.tokenizer.prepare_seq2seq_batch(
    inputs, max_length=args.max_source_length, return_tensors='pt'
)

print(inputs_encoding['input_ids'])

now = time.time()

model_ckp.model.eval()
with torch.no_grad():
    gen_output = model_ckp.model.generate(
        inputs_encoding['input_ids'],
        attention_mask=inputs_encoding['attention_mask'],
        use_cache=True,
        decoder_start_token_id = model_ckp.decoder_start_token_id,
        num_beams=model_ckp.eval_beams,
        max_length=model_ckp.eval_max_length,
        length_penalty=1.0
    )

print([model_ckp.tokenizer.decode(i) for i in gen_output])
print(time.time() - now)

tensor([[13959,     3, 21094,    12,  1566,    10, 32100,  1181,  8020,   235,
         32101,  2663,    13, 32102,  4338,  4316,     1]])
['Antipasto is an Italian dish.']
10.492921352386475


## Subject and Object Inverted
Using anternate aliases helps
'translate Graph to English: <H> 117852 Constance <R> follows <T> (117851) 2005 JE151'
'117852 Constance is followed by (117851) 2005 JE151.'
'translate Graph to English: <H> 117852 Constance <R> previous is <T> (117851) 2005 JE151'
'117852 Constance was preceded by (117851) 2005 JE151.'
    
'translate Graph to English: <H> Decius <R> child <T> Hostilian'
'Decius is a child of Hostilian.'
'translate Graph to English: <H> Decius <R>  has child <T> Hostilian'
'Decius has a child called Hostilian.'
    
## Hard Claim Syntax
These normally do not have aliases that are any easier to read
    
## Predicate Meaning Not Understood by Model

Artist | Aleksandr Vasilevitsj Vasjakin | conflict | Eastern Front | 1.6
Aleksandr Vasilevitsj Vasjakin is in the Eastern Front.
conflict -> participated in conflict
'Aleksandr Vasilevitsj Vasjakin participated in the conflict at the Eastern Front.'

Painting | Fresco depicting a menead carrying a thyrsus | movement | Ancient Roman mural painting | 2.0
Fresco depicting a menead carrying a thyrsus is a movement in the Ancient Roman mural painting.
artistic movement    
Fresco depicting a menead carrying a thyrsus is part of the artistic movement of the Ancient Roman mural painting.
    
## Redundant Claim Data
This is something that is emergent from how Wikidata stores information. For instance, an entity exists for a city, and another for its flag, which includes the flag's image. One is linked to the other by the flag predicate. This makes ontological sense, but no verbal sense, as one would say "This city has a flag" or "This city's flag is this city's flag", being either redundant or not quite communicating what the claim says. The same is true for things which are specifications/parts of others, like Israel's Cycling team of 1997 is part of Israel's Cycling team.
    
## Qualifiers needed
One would have to find a way of reliably tying qualifiers or descriptors to elements of the claim

## Vague predicate
Using alternative aliases works. Choosing the proper alias is tricky and depends on the context.
    


    